In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import joblib
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [2]:

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download("omw-1.4")
nltk.download("punkt_tab") 

[nltk_data] Downloading package punkt to C:\Users\1
[nltk_data]     Click\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\1
[nltk_data]     Click\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\1
[nltk_data]     Click\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\1
[nltk_data]     Click\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt_tab to C:\Users\1
[nltk_data]     Click\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [3]:
true = pd.read_csv("C:/Users/1 Click/Desktop/fake_news/True.csv")
fake = pd.read_csv("C:/Users/1 Click/Desktop/fake_news/Fake.csv")

In [4]:
print(true.head())

                                               title  \
0  As U.S. budget fight looms, Republicans flip t...   
1  U.S. military to accept transgender recruits o...   
2  Senior U.S. Republican senator: 'Let Mr. Muell...   
3  FBI Russia probe helped by Australian diplomat...   
4  Trump wants Postal Service to charge 'much mor...   

                                                text       subject  \
0  WASHINGTON (Reuters) - The head of a conservat...  politicsNews   
1  WASHINGTON (Reuters) - Transgender people will...  politicsNews   
2  WASHINGTON (Reuters) - The special counsel inv...  politicsNews   
3  WASHINGTON (Reuters) - Trump campaign adviser ...  politicsNews   
4  SEATTLE/WASHINGTON (Reuters) - President Donal...  politicsNews   

                 date  
0  December 31, 2017   
1  December 29, 2017   
2  December 31, 2017   
3  December 30, 2017   
4  December 29, 2017   


In [5]:
true["label"] = 1
fake["label"] = 0
news = pd.concat([true, fake], ignore_index=True)
news.drop(["title", "subject", "date"], axis=1, inplace=True)
print(news.head())

                                                text  label
0  WASHINGTON (Reuters) - The head of a conservat...      1
1  WASHINGTON (Reuters) - Transgender people will...      1
2  WASHINGTON (Reuters) - The special counsel inv...      1
3  WASHINGTON (Reuters) - Trump campaign adviser ...      1
4  SEATTLE/WASHINGTON (Reuters) - President Donal...      1


In [6]:
def simple_text_cleaner(text):
    if isinstance(text, list):
        text = " ".join(text)
    text = text.lower()
    words = word_tokenize(text)
    stop_words = set(stopwords.words("english"))
    clean_words = [w for w in words if w.isalpha() and w not in stop_words]
    lemmatizer = WordNetLemmatizer()
    final_words = [lemmatizer.lemmatize(w) for w in clean_words]
    return " ".join(final_words)

news["text"] = news["text"].apply(simple_text_cleaner)
print("hello")

hello


In [7]:
X = news["text"]
y = news["label"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)
print(X)

0        washington reuters head conservative republica...
1        washington reuters transgender people allowed ...
2        washington reuters special counsel investigati...
3        washington reuters trump campaign adviser geor...
4        reuters president donald trump called postal s...
                               ...                        
44893    century wire say reported earlier week unlikel...
44894    century wire say familiar theme whenever dispu...
44895    patrick henningsen century wireremember obama ...
44896    century wire say al jazeera america go history...
44897    century wire say predicted new year look ahead...
Name: text, Length: 44898, dtype: object


In [8]:
vectorizer = TfidfVectorizer()
Xv_train = vectorizer.fit_transform(X_train)
Xv_test = vectorizer.transform(X_test)
print(Xv_test)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2013628 stored elements and shape (13470, 80902)>
  Coords	Values
  (0, 193)	0.060898060765003544
  (0, 463)	0.042364110596508665
  (0, 514)	0.050451582555640646
  (0, 689)	0.039797762364910755
  (0, 1051)	0.037025950113790125
  (0, 1471)	0.07903079143611846
  (0, 1783)	0.06103467871560005
  (0, 2204)	0.06252126808938684
  (0, 3357)	0.04421928025763866
  (0, 3358)	0.03751008070484333
  (0, 3782)	0.04072331117166572
  (0, 4112)	0.04788932983217522
  (0, 4267)	0.0680380211775877
  (0, 5130)	0.02412631494914059
  (0, 5743)	0.3573675635455674
  (0, 6356)	0.04151711514881595
  (0, 6702)	0.10586195467546756
  (0, 7249)	0.08748212272846156
  (0, 7735)	0.08934189088639186
  (0, 7819)	0.050090236031409156
  (0, 8296)	0.04254885876144169
  (0, 8733)	0.052585674420699655
  (0, 8903)	0.06488448223675944
  (0, 10431)	0.09441413708504237
  (0, 10498)	0.05131782272339881
  :	:
  (13469, 44172)	0.09257110702482327
  (13469, 45037)	0.1023138

In [9]:
LR = LogisticRegression(max_iter=1000)  
LR.fit(Xv_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


In [10]:
prediction = LR.predict(Xv_test)
LR.score(Xv_test, y_test)

0.9847067557535264

In [13]:
from sklearn.tree import DecisionTreeClassifier
DTC=DecisionTreeClassifier()
DTC.fit(Xv_train,y_train)

,criterion,'gini'
,splitter,'best'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,None
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


In [16]:
prediction = DTC.predict(Xv_test)
DTC.score(Xv_test, y_test)

0.9953229398663697

In [15]:
from sklearn.ensemble import RandomForestClassifier
RFC=RandomForestClassifier()
RFC.fit(Xv_train,y_train)


,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [17]:
prediction = RFC.predict(Xv_test)
RFC.score(Xv_test, y_test)

0.9920564216778025

In [19]:
# %%
joblib.dump(vectorizer, "vectorizer.jb")
joblib.dump(LR, "lr_model.jb")
print("✅ Model training complete. Files saved: vectorizer.jb, lr_model.jb")




✅ Model training complete. Files saved: vectorizer.jb, lr_model.jb
